In [1]:
import matplotlib.pyplot as plt
from pulp import *
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
# import gurobipy
from gurobipy import Model, GRB
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
import pickle as pkl
import math
from tqdm import tqdm


In [2]:
with open('.cache/int_basins_PAs.pkl', 'rb') as f:
    int_basins_PAs = pkl.load(f)
int_basins_PAs['HYBAS_ID'] = int_basins_PAs['HYBAS_ID'].astype(int)
int_basins_PAs['MAIN_BAS'] = int_basins_PAs['MAIN_BAS'].astype(int)

In [9]:
with open('.cache/batched_output/data.pkl', 'rb') as f:
    final_gdf = pkl.load(f)
    basins = pkl.load(f)

basins_set = set(basins)

In [43]:
basins_gdf = final_gdf.copy()
other_gdf_data = {}
other_basins = []
other_area = []
other_main_basins = []

In [44]:
basins_gdf

,HYBAS_ID,MAIN_BAS,PROTECTED,geometry,AREA,uniqueID,id_no,sci_name
0,6120000010,6030000010,0.0,"MULTIPOLYGON (((-78.95126 9.54998, -78.95750 9...",203.286676,1,"{49829891, 185872, 186393, 149249053, 188471, ...","{Centropomus pectinatus, Ctenogobius stigmatic..."
1,6120000010,6030000010,1.0,"POLYGON ((-79.26720 9.42725, -79.27030 9.43109...",17.915474,0,"{138525825, 49829891, 191108, 185989, 185990, ...","{Brycon chagrensis, Dajaus monticola, Synbranc..."
2,6120000020,6030000010,0.0,"POLYGON ((-79.10000 9.40417, -79.10035 9.40659...",70.898202,2,"{138525825, 49829891, 191108, 185989, 185990, ...","{Astroblepus pirrense, Dajaus monticola, Chaet..."
3,6120000030,6030000010,0.0,"MULTIPOLYGON (((-79.09583 9.33333, -79.10833 9...",114.441867,3,"{49829891, 185872, 186393, 149249053, 188471, ...","{Roeboides dayi, Centropomus pectinatus, Cteno..."
4,6120000040,6030000010,0.0,"POLYGON ((-79.09167 9.30000, -79.09082 9.30332...",119.254719,4,"{49829891, 185872, 186393, 149249053, 53876256...","{Roeboides dayi, Eretmobrycon bayano, Awaous t..."
...,...,...,...,...,...,...,...,...
153722,6121361740,6030021870,0.0,"POLYGON ((-69.10242 -54.85798, -69.10591 -54.8...",6.679227,160579,"{176549691, 176561197}","{Aplochiton taeniatus, Galaxias platei}"
153723,6121361740,6030021870,1.0,"POLYGON ((-69.10939 -54.85073, -69.12712 -54.8...",187.589412,160580,"{176549691, 176561197}","{Aplochiton taeniatus, Galaxias platei}"
153724,6121361750,6030021870,0.0,"POLYGON ((-67.90417 -55.07917, -67.90833 -55.0...",172.350953,160581,{176549691},{Aplochiton taeniatus}
153725,6121361760,6030021870,0.0,"POLYGON ((-67.28333 -55.09167, -67.29891 -55.0...",134.402067,160582,{176549691},{Aplochiton taeniatus}


In [45]:
for index, row in tqdm(int_basins_PAs.iterrows()):
    basin_origid = row['HYBAS_ID']
    protected = row['PROTECTED']
    area = row['AREA']
    main_origid = row['MAIN_BAS']

    if (basin_origid, protected) not in basins_set:
        other_basins.append((basin_origid, protected))
        new_row = row.copy()
        new_row['id_no'] = set()  # Assuming you want to add NaN or NA values
        new_row['sci_name'] = set()
        basins_gdf.loc[len(basins_gdf)] = new_row
        # basins_gdf = pd.concat([basins_gdf, new_row], ignore_index=True)
        # basins_gdf = basins_gdf.concat(new_row, ignore_index=True)
        other_area.append(area)
        other_main_basins.append(main_origid)

160584it [01:51, 1439.04it/s] 


In [69]:
# basins_gdf

# with open('analysis/basins_gdf.pkl', 'wb') as f:
#     pkl.dump(basins_gdf, f)

In [56]:
counts = basins_gdf.groupby(['MAIN_BAS']).size().reset_index(name='Count')
counts.sort_values(by=['Count'])

,MAIN_BAS,Count
25,6030040020,5
17,6030016980,189
5,6030006540,389
9,6030007960,656
0,6030000010,747
7,6030007010,991
24,6030032290,2125
10,6030008320,2321
23,6030029280,2610
12,6030009770,2673


In [57]:
with open('.cache/fish_def.pkl', 'rb') as f:
    fish_def = pkl.load(f)
fish_def

,Shape_Leng,Shape_Area,id_no,sci_name,presence,origin,seasonal,compiler,yrcompiled,citation,...,family,genus,category,marine,terrestial,freshwater,layer,path,geometry,geometry_buffer
0,285.917819,850.025112,49830062,Pimelodus ornatus,1,1,1,"Salvador, G.N.",2020,IUCN (International Union for Conservation of ...,...,PIMELODIDAE,Pimelodus,LC,false,false,true,FW_FISH_PART1,/Users/im298/Library/CloudStorage/Box-Box/Data...,"MULTIPOLYGON (((-55.50472 -24.99944, -55.50665...","MULTIPOLYGON (((-69.16250 -14.76250, -69.16133..."
1,5.283163,0.704958,186819,Phenacogaster calverti,1,1,1,IUCN Freshwater Biodiversity Unit,2021,IUCN (International Union for Conservation of ...,...,CHARACIDAE,Phenacogaster,LC,false,false,true,FW_FISH_PART2,/Users/im298/Library/CloudStorage/Box-Box/Data...,"MULTIPOLYGON (((-38.36888 -7.40057, -38.36029 ...","MULTIPOLYGON (((-38.09583 -5.81250, -38.09466 ..."
2,12.931535,11.556144,167658,Lithoxus bovallii,1,1,1,Mala Ram,2007,"Red List Index (Sampled Approach), Zoological ...",...,LORICARIIDAE,Lithoxus,LC,false,false,true,FW_FISH_PART2,/Users/im298/Library/CloudStorage/Box-Box/Data...,"POLYGON ((-61.33134 2.02992, -61.45747 2.23174...","POLYGON ((-61.33133 2.02992, -61.45747 2.23174..."
3,9.057081,0.596283,167762,Rineloricaria wolfei,1,1,1,Marcelo Tognelli,2015,IUCN,...,LORICARIIDAE,Rineloricaria,LC,false,false,true,FW_FISH_PART1,/Users/im298/Library/CloudStorage/Box-Box/Data...,"MULTIPOLYGON (((-69.42973 -12.97721, -69.43277...","MULTIPOLYGON (((-69.42973 -12.97721, -69.43277..."
4,236.835554,151.837336,164584163,Moenkhausia mikia,1,1,1,The Jolly Geographer,2022,IUCN (International Union for Conservation of ...,...,CHARACIDAE,Moenkhausia,LC,false,false,true,FW_FISH_PART2,/Users/im298/Library/CloudStorage/Box-Box/Data...,"MULTIPOLYGON (((-59.30893 -4.33661, -59.30368 ...","MULTIPOLYGON (((-71.56667 1.12500, -71.57083 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903,56.180256,3.537734,186440,Pimelodella avanhandavae,1,1,1,IUCN Freshwater Biodiversity Unit,2020,IUCN (International Union for Conservation of ...,...,HEPTAPTERIDAE,Pimelodella,LC,false,false,true,FW_FISH_PART1,/Users/im298/Library/CloudStorage/Box-Box/Data...,"MULTIPOLYGON (((-54.04391 -25.30470, -54.04167...","MULTIPOLYGON (((-44.40057 -21.46741, -44.40057..."
4904,14.201999,0.714835,176145483,Auchenipterus demerarae,1,1,1,"Calegari, B.B.",2020,IUCN (International Union for Conservation of ...,...,AUCHENIPTERIDAE,Auchenipterus,LC,false,false,true,FW_FISH_PART1,/Users/im298/Library/CloudStorage/Box-Box/Data...,"MULTIPOLYGON (((-58.67855 2.16049, -58.68009 2...","MULTIPOLYGON (((-58.36446 6.04640, -58.36446 6..."
4905,9.431438,1.073396,49829985,Bryconamericus osgoodi,1,1,1,Marcelo Tognelli,2015,IUCN,...,CHARACIDAE,Bryconamericus,LC,false,false,true,FW_FISH_PART1,/Users/im298/Library/CloudStorage/Box-Box/Data...,"MULTIPOLYGON (((-72.95612 -11.44223, -72.95221...","MULTIPOLYGON (((-72.95612 -11.44223, -72.95221..."
4906,12.047454,0.779781,49830327,Bujurquina megalospilus,1,1,1,Marcelo Tognelli,2015,IUCN,...,CICHLIDAE,Bujurquina,LC,false,false,true,FW_FISH_PART2,/Users/im298/Library/CloudStorage/Box-Box/Data...,"MULTIPOLYGON (((-74.71250 -9.42917, -74.71307 ...","MULTIPOLYGON (((-72.71421 -9.41215, -72.71004 ..."


In [94]:
species = [] # species id -> {id_no: , sci_name: }
species_orig_to_id = {} # id_no -> species id
for index, row in tqdm(fish_def.iterrows()):
    id_no = row['id_no']
    sci_name = row['sci_name']
    species_orig_to_id[id_no] = len(species)
    species.append({'id_no': id_no, 'sci_name':sci_name})


4908it [00:01, 3164.24it/s] 


In [95]:
num_species = len(species) # 4908
num_basins = len(basins_gdf) # 160584

basins_info = [None]*num_basins #id -> {hybas_id: , main_id: , protected: , area: }
protected_basins = set()
basin_species = [None]*num_basins # id -> set of species ids

main_to_basin = {}


for index, row in tqdm(basins_gdf.iterrows()):
    hybas_id = row['HYBAS_ID']
    main_id = row['MAIN_BAS']
    protected = row['PROTECTED']
    area = row['AREA']
    id = row['uniqueID']
    fish = row['id_no']

    basins_info[id] = {'hybas_id': hybas_id, 'main_id': main_id, 'protected': protected, 'area': area}
    if protected == 1:
        protected_basins.add(id)
    id_set = main_to_basin.get(main_id, set())
    id_set.add(id)
    main_to_basin[main_id] = id_set

    basin_species[id] = {species_orig_to_id[f] for f in fish}
    assert len(basin_species[id]) == len(fish)

0it [00:00, ?it/s]

160584it [00:11, 13522.35it/s]


In [83]:
num_protected = len(protected_basins) # 36565
total_area = sum([basins_info[id]['area'] for id in range(num_basins)]) # 17765548.969
protected_area = sum([basins_info[id]['area'] for id in protected_basins]) # 2996293.72
print(protected_area/total_area) # 0.16865753630596783

0.16865753630596783


In [98]:
species_basin = [] # species id -> set of basins
for i in range(num_species):
    species_basin.append(set())
for id in tqdm(range(num_basins)):
    fishes = basin_species[id]
    for fish in fishes:
        species_basin[fish].add(id)

  0%|          | 0/160584 [00:00<?, ?it/s]

100%|██████████| 160584/160584 [00:03<00:00, 42764.43it/s] 


In [101]:
# with open('analysis/data.pkl', 'wb') as f:
#     pkl.dump(basins_info, f)
#     pkl.dump(protected_basins, f)
#     pkl.dump(main_to_basin, f)
#     pkl.dump(num_basins, f)

#     pkl.dump(species, f)
#     pkl.dump(species_orig_to_id, f)
#     pkl.dump(num_species, f)

#     pkl.dump(basin_species, f)
#     pkl.dump(species_basin, f)

#     pkl.dump(num_protected, f)
#     pkl.dump(total_area, f)
#     pkl.dump(protected_area, f)


In [103]:
P = np.zeros((num_species, num_basins))
habitat_area_percentage = [] # species id -> habitat area
for j in tqdm(range(num_species)):
    habitat_area = sum([basins_info[i]['area'] for i in species_basin[j]])
    habitat_area_percentage.append(habitat_area/total_area)
    # print(i, habitat_area)

    for i in species_basin[j]:
        P[j][i] = basins_info[i]['area']/habitat_area

100%|██████████| 4908/4908 [00:12<00:00, 392.19it/s]


In [106]:
# with open('analysis/P.pkl', 'wb') as f:
#     pkl.dump(P, f)
#     pkl.dump(habitat_area_percentage, f)

In [2]:
with open('analysis/data.pkl', 'rb') as f:
    basins_info = pkl.load(f)
    protected_basins = pkl.load(f)
    main_to_basin = pkl.load(f)
    num_basins = pkl.load(f)

    species = pkl.load(f)
    species_orig_to_id = pkl.load(f)
    num_species = pkl.load(f)

    basin_species = pkl.load(f)
    species_basin = pkl.load(f)

    num_protected = pkl.load(f)
    total_area = pkl.load(f)
    protected_area = pkl.load(f)

In [3]:
with open('analysis/P.pkl', 'rb') as f:
    P = pkl.load(f)
    habitat_area_percentage = pkl.load(f)

In [37]:
basin_totalscore = np.sum(P, axis=0) # basin id -> total score

In [7]:
with open('analysis/basins_gdf.pkl', 'rb') as f:
    basins_gdf = pkl.load(f)

In [7]:
main_to_total_area = {}
main_to_protected_area = {}
for main_id in main_to_basin:
    ids = main_to_basin[main_id]
    protected_ids = [id for id in ids if id in protected_basins]

    main_num_basins = len(ids)
    main_num_protected = len(protected_ids)

    main_total_area = sum([basins_info[id]['area'] for id in ids])
    main_protected_area = sum([basins_info[id]['area'] for id in protected_ids])

    main_to_total_area[main_id] = main_total_area
    main_to_protected_area[main_id] = main_protected_area

    # print(main_id)
    # print('total area', main_total_area)
    # print('protected area', main_protected_area)
    # print(main_protected_area/main_total_area)
    # print()

print('big basin id, total area, protected area percentage')
for main_id in sorted(main_to_basin.keys(), key=lambda x: -main_to_total_area[x]):
    print(main_id, ',', main_to_total_area[main_id], ',', main_to_protected_area[main_id]/main_to_total_area[main_id])

big basin id, total area, protected area percentage
6030007000 , 5888268.700897417 , 0.24144322011993297
6030016970 , 2639953.6736192135 , 0.056070611689446485
6030004470 , 937351.7793890522 , 0.37797407771848596
6030007950 , 769445.060774707 , 0.09837876877096054
6030018860 , 706407.9938055158 , 0.07146915886866176
6030014330 , 635917.4067588404 , 0.04763254478292298
6030011780 , 634841.534826135 , 0.12292957211690672
6030011790 , 574104.7703240376 , 0.07516143181818907
6030004480 , 561988.5307751594 , 0.30314930050749517
6030026140 , 414714.60272501566 , 0.0310812584233197
6030018850 , 400818.8071181148 , 0.10254534717799474
6030017370 , 392345.94744435 , 0.017651497699383283
6030021870 , 341976.55334214284 , 0.3573338400298418
6030009760 , 331566.2317958669 , 0.09968422443669203
6030040470 , 326030.8432134486 , 0.08727029372957301
6030040460 , 307865.61204628693 , 0.061812313449493325
6030029280 , 291264.17728403583 , 0.07397127892401195
6030009770 , 285929.5316977642 , 0.0540165637

In [10]:
final_gdf

,HYBAS_ID,MAIN_BAS,PROTECTED,geometry,AREA,uniqueID,id_no,sci_name
0,6120000010,6030000010,0.0,"MULTIPOLYGON (((-78.95126 9.54998, -78.95750 9...",203.286676,1,"{49829891, 185872, 186393, 149249053, 188471, ...","{Chaetostoma fischeri, Ctenogobius fasciatus, ..."
1,6120000010,6030000010,1.0,"POLYGON ((-79.26720 9.42725, -79.27030 9.43109...",17.915474,0,"{138525825, 49829891, 191108, 185989, 185990, ...","{Poecilia gillii, Chaetostoma fischeri, Brachy..."
2,6120000020,6030000010,0.0,"POLYGON ((-79.10000 9.40417, -79.10035 9.40659...",70.898202,2,"{138525825, 49829891, 191108, 185989, 185990, ...","{Poecilia gillii, Chaetostoma fischeri, Brachy..."
3,6120000030,6030000010,0.0,"MULTIPOLYGON (((-79.09583 9.33333, -79.10833 9...",114.441867,3,"{49829891, 185872, 186393, 149249053, 188471, ...","{Chaetostoma fischeri, Ctenogobius fasciatus, ..."
4,6120000040,6030000010,0.0,"POLYGON ((-79.09167 9.30000, -79.09082 9.30332...",119.254719,4,"{49829891, 185872, 186393, 149249053, 53876256...","{Chaetostoma fischeri, Ctenogobius fasciatus, ..."
...,...,...,...,...,...,...,...,...
153722,6121361740,6030021870,0.0,"POLYGON ((-69.10242 -54.85798, -69.10591 -54.8...",6.679227,160579,"{176549691, 176561197}","{Galaxias platei, Aplochiton taeniatus}"
153723,6121361740,6030021870,1.0,"POLYGON ((-69.10939 -54.85073, -69.12712 -54.8...",187.589412,160580,"{176549691, 176561197}","{Galaxias platei, Aplochiton taeniatus}"
153724,6121361750,6030021870,0.0,"POLYGON ((-67.90417 -55.07917, -67.90833 -55.0...",172.350953,160581,{176549691},{Aplochiton taeniatus}
153725,6121361760,6030021870,0.0,"POLYGON ((-67.28333 -55.09167, -67.29891 -55.0...",134.402067,160582,{176549691},{Aplochiton taeniatus}


In [17]:
nontrivial_basins = set(basins_gdf['uniqueID'][:153727])

In [23]:
trivial_basins = set(i for i in range(num_basins) if i not in nontrivial_basins) # 6857 basins

In [35]:
non_protected_basins = set(i for i in range(num_basins) if i not in protected_basins) # 124019 basins

In [27]:
protect_trivial = trivial_basins.intersection(protected_basins) # 1616 basins

In [3]:
area_bound = 0.3
max_additional_area = area_bound * total_area - protected_area

In [5]:
print(protected_area/total_area)

0.16865753630596783


In [2]:
def solveLP_max_totalScore():
    m = Model("max total score")
    
    # m.update()
    # try:
    #     num_threads = m.getAttr('Threads')
    #     print(f"Gurobi is configured to use {num_threads} threads (cores).")
    # except Exception as e:
    #     print(f"Error retrieving the number of threads: {e}")

    vars = {}
    for i in non_protected_basins:
        x = m.addVar(vtype=GRB.CONTINUOUS, name="x"+str(i))
        vars[i] = x
        m.addConstr(vars[i] >= 0, "c0_"+str(i))
        m.addConstr(vars[i] <= 1, "c1_"+str(i))

    area = sum(basins_info[i]['area'] * vars[i] for i in non_protected_basins)
    m.addConstr(area <= max_additional_area, name='c_area')

    score = sum(basin_totalscore[i] * vars[i] for i in non_protected_basins)
    m.setObjective(score, GRB.MAXIMIZE)

    m.optimize()

    sol = {}
    if m.status == GRB.OPTIMAL:
        print('Optimal solution found:')
        for v in m.getVars():
            i = int(v.varName[1:])
            sol[i] = v.x
            print(f'{v.varName} = {v.x}')
        # for i in non_protected_basins:
        #     sol[i] = f"v.x{i}" # modify the code here
    else:
        print('No optimal solution found.')

    return sol, m.ObjVal

sol_max_totalscore, max_totalscore = solveLP_max_totalScore()

GurobiError: Unable to open Gurobi license file '/home/fs01/ym277/gurobi.lic'

In [68]:
# sol_max_totalscore
for i in sol_max_totalscore:
    sol_i = sol_max_totalscore[i]
    # if 0 < sol_i < 1:
    #     print(i, format(sol_i, '.10f'))
    if sol_i>0:
        if i in trivial_basins:
            print(i)

In [67]:
P.shape

(4908, 160584)

In [2]:
def solveLP_max_theta():
    m = Model("max theta")

    vars = {}
    for i in range(num_basins):
        x = m.addVar(vtype=GRB.CONTINUOUS, name="x"+str(i))
        vars[i] = x
        m.addConstr(vars[i] >= (0 if i in non_protected_basins else 1), "c0_"+str(i))
        m.addConstr(vars[i] <= 1, "c1_"+str(i))

    area = sum(basins_info[i]['area'] * vars[i] for i in non_protected_basins)
    m.addConstr(area <= max_additional_area, name='c_area')

    theta = m.addVar(vtype=GRB.CONTINUOUS, name='theta')
    # scores = []
    for j in range(num_species):
        score = sum(P[j][i] * vars[i] for i in range(num_basins))
        m.addConstr(score >= theta, 'theta_species_'+str(j))
    m.setObjective(theta, GRB.MAXIMIZE)

    m.optimize()

    sol = {}
    if m.status == GRB.OPTIMAL:
        print('Optimal solution found:')
        for v in m.getVars():
            i = int(v.varName[1:])
            sol[i] = v.x
            print(f'{v.varName} = {v.x}')
        # for i in non_protected_basins:
        #     sol[i] = f"v.x{i}" # modify the code here
    else:
        print('No optimal solution found.')

    return sol, m.ObjVal

sol_max_theta, max_theta = solveLP_max_theta()

GurobiError: Unable to open Gurobi license file '/home/fs01/ym277/gurobi.lic'

In [ ]:
def solveLP_fixed_theta_max_totalScore(theta):
    return